In [4]:
# pip install -U spacy
# python -m spacy download en_core_web_sm
# %pip install openai

import pandas as pd
import numpy as np
from openai import OpenAI
import json
from collections import OrderedDict

In [2]:
philly_reviews = pd.read_csv('philadelphia_restaurant_reviews_sampled(in).csv')

In [3]:
client = OpenAI()

annotated_reviews = OrderedDict()

for review in philly_reviews:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a restaurant review annotator. Categorize the following text so I can evaluate the restaurant accurately. Then for each category give the restaurant a score from 1-5, 1 being bad and 5 being excellent for their performance in each category. Include every single dish mentioned in the review. Return the output in three column JSON format with category and score."},
            {"role": "user", "content": review['text']},
        ],
        max_tokens=256
    )
    enriched = response['choices'][0]['message']['content']
    
    try:
        enriched = json.loads(enriched)
    except:
        print('Error parsing JSON')
        print(enriched)
        continue
    
    if review['business_id'] not in annotated_reviews:
        annotated_reviews[review['business_id']] = OrderedDict()
    annotated_reviews[review['business_id']][review['review_id']] = enriched 

# TODO: Save annotated_reviews to a file
# with open('annotated_reviews.json', 'w') as f:
#     json.dump(annotated_reviews, f)

# TODO: Aggregate annotated_reviews to get average scores for each category and business


KeyboardInterrupt

